In [2]:
from pyrte_rrtmgp.external_data_helpers import download_dyamond2_data
from datetime import datetime

# Download the data
downloaded_files = download_dyamond2_data(datetime(2020, 2, 1, 9), compute_gas_optics=False, data_dir="GEOS-DYAMOND2-data")

In [ ]:
import xarray as xr
from pyrte_rrtmgp.constants import HELMERT1
from pyrte_rrtmgp.data_types import CloudOpticsFiles
from pyrte_rrtmgp import rrtmgp_cloud_optics
import dask
from dask.distributed import Client

min_lev_liquid = 107
min_lev_ice = 78

# # Create a dask client with 10 workers
# client = Client(n_workers=10)
# print(f"Dask dashboard available at: {client.dashboard_link}")

atmosphere = xr.open_mfdataset(
    "GEOS-DYAMOND2-data/*inst_01hr_3d_*.nc4",
    drop_variables=[
        "anchor",
        "cubed_sphere",
        "orientation",
        "contacts",
        "corner_lats",
        "corner_lons",
    ],
).isel(lev=slice(min_lev_ice,)).rename({"lev": "layer"}).isel(Ydim=slice(0, 300)).chunk({"Xdim": 2880, "Ydim": 1, "nf": -1, "layer": -1})

#
# By default the dask arrays are contiguous in Xdim and Ydim (first two dimensions)
#   This will work for computing optics but not for computing fluxes,
#   where chunks need to include all layers/levels

# Need to convert LWP/IWP to g/m2 and rel/rei to microns
atmosphere["lwp"] = (atmosphere["DELP"] * atmosphere["QL"]) * 1000 / HELMERT1
atmosphere["iwp"] = (atmosphere["DELP"] * atmosphere["QI"]) * 1000 / HELMERT1
atmosphere["rel"] = atmosphere["RL"] * 1e6
atmosphere["rei"] = atmosphere["RI"] * 1e6

needed_vars = ["lwp", "iwp", "rel", "rei"]
cloud_optics_lw = rrtmgp_cloud_optics.load_cloud_optics(cloud_optics_file=CloudOpticsFiles.LW_BND)
tau = cloud_optics_lw.compute_cloud_optics(atmosphere[needed_vars], problem_type="absorption", add_to_input=False)

tau.to_netcdf("cloud_optics_result.nc")
